In [2]:
# using Pkg;
# #Pkg.add("DFTK")
# Pkg.update()

using ParallelTemperingMonteCarlo
using DelimitedFiles
using StaticArrays
using Random#, Plots

# Pkg.add("BenchmarkTools")
using BenchmarkTools

┌ Warning: scipy python package not found. WignerSeitz.wignerseitz is nonfunctional.
└ @ Brillouin.WignerSeitz ~/.julia/packages/Brillouin/nD01A/src/WignerSeitz.jl:45


┌ Warning: scipy python package not found. WignerSeitz.wignerseitz is nonfunctional.
└ @ Brillouin.WignerSeitz /home/grayseff/.julia/packages/Brillouin/nD01A/src/WignerSeitz.jl:45


In [3]:
genvector1(max_displacement) = max_displacement * (rand(-0.5:0.001:0.5,3))

genvector2(max_displacement) = SVector((rand()-0.5)*max_displacement,(rand()-0.5)*max_displacement,(rand()-0.5)*max_displacement)
@benchmark genvector1(rand())

BenchmarkTools.Trial: 10000 samples with 585 evaluations.
 Range (min … max):  200.342 ns …   9.600 μs  ┊ GC (min … max): 0.00% … 94.68%
 Time  (median):     256.068 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   282.654 ns ± 292.700 ns  ┊ GC (mean ± σ):  3.61% ±  3.54%

  █▆▅▄▃▃▂▁▇▅▄▃▄▄▃▃▄▂▃▄▃▂▂▁      ▂▂▂▁                            ▂
  █████████████████████████▇█████████▇▇▇██▇███▇█▆▅▆▅▄▆▄▄▅▂▆▄▄▄▄ █
  200 ns        Histogram: log(frequency) by time        591 ns <

 Memory estimate: 160 bytes, allocs estimate: 2.

In [4]:
@benchmark genvector2(rand())

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):   9.309 ns … 249.550 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.912 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.131 ns ±   6.090 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂  ▁        █   ▄                                             
  █▇██▇▄▂▂▁▁▁▅█▄▂▂█▇▂▂▁▁▂▇▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  9.31 ns         Histogram: frequency by time           22 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [5]:
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 1008000 #default 20% equilibration cycles on top

mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

In [6]:
mc_states[2]

MCState{Float64, 13, SphericalBC{Float64}}(5.1911689669640735, 60829.26355752399, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[5.339202509675499, 1.7547286276557148, 0.9552945639202041], [3.8235622875401982, -4.036693324695786, 1.2586922223528678], [3.8432525502537267, -1.2169635315645375, -4.030786193502644], [1.8515104475315411, 4.369051328639683, -3.1594504692142205], [1.8196441394724319, -0.19336921007065874, 5.399104677171738], [0.6008527770958447, 5.001668105430829, 2.668542641160762], [-5.339209779512827, -1.754731061649526, -0.9552951725503948], [-0.6008496122188508, -5.001669753738395, -2.668542209586626], [-1.851508499387977, -4.369053575565342, 3.1594501498809775], [-1.8196475277773039, 0.19336884278473446, -5.399104511708638], [-3.823559204867802, 4.036697652557049, -1.2586918476896083], [-3.8432544820617607, 1.2169644600549232, 4.030785366187147], [4.394257284152319e-6, 1.4401613097476533e-6, 7.841053896189476e-7]], SphericalBC{Float64}(101.06969058367278

In [7]:
function testtime(mc_states)
    for state in mc_states
        ran = rand(1:length(state.config.pos))
        trial_pos =atom_displacement(state.config.pos[ran],state.max_displ[1],state.config.bc)
    end
end

testtime (generic function with 1 method)

In [8]:
@benchmark testtime(mc_states)

BenchmarkTools.Trial: 10000 samples with 195 evaluations.
 Range (min … max):  482.564 ns …   4.048 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     595.128 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   608.579 ns ± 183.052 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇▄▃▃▁   █▄▄▃▂▁▁▁▁▃▁                                          ▂
  ███████▇▆████████████▇▇███▇▇▇▇▇▇▇▆█▇▇▆▆▆▇▆▅▇▇▆▇▆▆▅▆▆▄▅▅▆▄▅▃▆▅ █
  483 ns        Histogram: log(frequency) by time       1.36 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [9]:
# rand(1:n_atoms,n_traj)
# V = [1 2 3; 4 5 6; 7 8 9]
# V[:,1]
function atom_disp(mc_state,index)
    trial_pos = atom_displacement(mc_state.config.pos[index],mc_state.max_displ[1],mc_state.config.bc)
    return trial_pos
end
function atom_disp!(mc_state,index,trial_pos::SVector)
    trial_pos = atom_displacement(mc_state.config.pos[index],mc_state.max_displ[1],mc_state.config.bc)
    return trial_pos
end

indices = rand(1:n_atoms,n_traj)

# trial_pos = atom_disp.(mc_states,indices)


32-element Vector{Int64}:
 11
 12
  8
  9
  5
  8
 10
  8
  7
 10
  ⋮
 10
  8
  5
  8
  4
  5
  2
 10
  3

In [10]:
@benchmark begin   
    indices = rand(1:n_atoms,n_traj)
    trial_positions = atom_disp.($mc_states,indices)
end


BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.060 μs … 635.600 μs  ┊ GC (min … max): 0.00% … 98.46%
 Time  (median):     2.450 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.656 μs ±   6.742 μs  ┊ GC (mean ± σ):  2.36% ±  0.98%

    █       ▁                                                  
  ▃▅█▇▄▃▄▅▄██▅▆▅▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂ ▃
  1.06 μs         Histogram: frequency by time        9.96 μs <

 Memory estimate: 1.30 KiB, allocs estimate: 5.

In [11]:
@benchmark begin 
    indices = rand(1:n_atoms,n_traj)
    trial_positions =  [@SVector zeros(3) for i in 1:n_traj]
    atom_disp!.($mc_states,indices,trial_positions)
end

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.420 μs … 522.920 μs  ┊ GC (min … max): 0.00% … 97.89%
 Time  (median):     1.980 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.873 μs ±   8.620 μs  ┊ GC (mean ± σ):  5.05% ±  1.70%

   ▇  ▁█                                                       
  ▆█▇▆██▆▂▂▂▁▁▁▁▁▁▂▂▂▃▄▄▄▅▅▄▄▄▃▃▂▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  1.42 μs         Histogram: frequency by time        6.57 μs <

 Memory estimate: 2.23 KiB, allocs estimate: 8.

In [12]:
function generate_displacements(mc_states,mc_params)
    indices=rand(1:mc_params.n_atoms,mc_params.n_traj)
    trial_positions = atom_disp.(mc_states,indices)
    return indices,trial_positions
end

generate_displacements (generic function with 1 method)

In [13]:
@benchmark generate_displacements(mc_states,mc_params)

BenchmarkTools.Trial: 10000 samples with 181 evaluations.
 Range (min … max):  575.691 ns … 129.767 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     780.110 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.328 μs ±   2.387 μs  ┊ GC (mean ± σ):  7.20% ± 5.43%

  █▃█▅▂▄▂▁▁▁▁ ▂▄▄▄▃▄▄▄▃▂▂▁▁▁                                    ▂
  ████████████████████████████▇▇▇▇▇▆▇▆▇▆▅▅▅▆▄▄▆▄▃▅▅▄▄▄▄▃▄▄▄▃▃▄▄ █
  576 ns        Histogram: log(frequency) by time       5.08 μs <

 Memory estimate: 1.23 KiB, allocs estimate: 3.

In [15]:
@benchmark dimer_energy_atom(1, mc_states[1].dist2_mat[1,:], pot)

BenchmarkTools.Trial: 10000 samples with 197 evaluations.
 Range (min … max):  463.452 ns …  32.754 μs  ┊ GC (min … max): 0.00% … 96.84%
 Time  (median):     604.569 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   669.937 ns ± 649.876 ns  ┊ GC (mean ± σ):  1.99% ±  2.16%

  █      ▃                                                       
  █▇▃▂▂▅▃█▆▆▄▃▂▂▃▂▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  463 ns           Histogram: frequency by time         1.56 μs <

 Memory estimate: 192 bytes, allocs estimate: 3.

In [14]:
@benchmark dimer_energy_atom(1, mc_states[1].dist2_mat[:,1], pot)

BenchmarkTools.Trial: 10000 samples with 204 evaluations.
 Range (min … max):  379.902 ns …  58.784 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     500.490 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   566.402 ns ± 839.067 ns  ┊ GC (mean ± σ):  2.11% ± 2.16%

  ██     ▄                                                       
  ██▄▃▇▃▂█▅▅▆▇▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  380 ns           Histogram: frequency by time         1.32 μs <

 Memory estimate: 192 bytes, allocs estimate: 3.